In [1]:
import numpy as np
import xarray as xr
import xdggs
import healpy

In [ ]:
ZOOM = 8
N = int(10e6)
da = xr.DataArray(
    data = np.arange(N),
    dims = ('cell_ids',),
    coords = {
        'cell_ids': ('cell_ids', np.arange(N),{"grid_name": "healpix", "level": ZOOM, "indexing_scheme": "nested"})},
)


def sel_and_decode(da, cell_ids):
    da = da.copy()

    return da.sel(
        cell_ids=cell_ids,
    ).dggs.decode()

def get_cell_ids(lon_lat_list:list[tuple[float, float]], zoom: int):
    nsides = 2**zoom

    return healpy.query_polygon(
        nsides,
        [lon_lat_to_xyz(lon_lat) for lon_lat in lon_lat_list],
        inclusive=True,
        nest=True,
    )
def lon_lat_to_xyz(lon_lat: tuple[float, float]) -> tuple[float, float, float]:
    # https://en.wikipedia.org/wiki/Spherical_coordinate_system
    r = 1
    lat, lon = lon_lat
    lat = np.radians(lat)
    lon = np.radians(lon)
    x = r * np.cos(lat) * np.cos(lon)
    y = r * np.cos(lat) * np.sin(lon)
    z = r * np.sin(lat)
    return x, y, z


def get_aus_bbox():
    return (113.338953078, -43.6345972634, 153.569469029, -10.6681857235)


MIN_LON, MIN_LAT, MAX_LON, MAX_LAT = get_aus_bbox()

POLYGON = [
    (MIN_LAT, MIN_LON),
    (MIN_LAT, MAX_LON),
    (MAX_LAT, MAX_LON),
    (MAX_LAT, MIN_LON),
]

cells = get_cell_ids(POLYGON, ZOOM)
cells1 = np.array([1344, 1345, 1347, 1348, 1349, 1350, 2492, 2493, 2495, 2530, 2531,
       2536, 2537, 2538, 2539, 2540, 2542, 2543], dtype=np.uint64)

da_truth = sel_and_decode(da, cells)
da1 = sel_and_decode(da, cells1)






ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [30]:
cells

array([345083, 346449])

In [31]:
da_truth.dggs.explore()

Map(custom_attribution='', layers=(SolidPolygonLayer(filled=True, get_fill_color=arro3.core.ChunkedArray<Fixed…

In [ ]:
da1.dggs.explore()

In [ ]:
da.dggs.grid_info

In [ ]:
g = da.dggs.downscale(3, np.max)

In [ ]:
def upscale(obj, *, offset, grid_info):
    """Upscale a dataset by copying data from the parent cells to the child cells."""
    if not grid_info.nest:
        raise NotImplementedError("Upscaling is only supported for nested Healpix grids.")

    if offset == 0:
        return obj
    
    obj = obj.copy()

    first_child_cells = obj.cell_ids * (4**offset)

    obj.cell_ids = first_child_cells

In [ ]:
children = da.dggs.children
da["cell_ids"] =  da.cell_ids * (4**2)


In [ ]:
da = da.sel(cell_ids = children)
da.ffill("cell_ids")

In [ ]:
g.groups

In [ ]:
bins = np.floor(da.cells / (coarsen*4))

In [ ]:
da.groupby(bins).mean()

In [ ]:
da.groupby(cells=slice(None, None, 4))